In [1]:
import facebook
import requests
import json
import csv
from kafka import KafkaClient
from kafka import SimpleProducer
import sys

In [2]:
kafka = KafkaClient('localhost:9092')
producer = SimpleProducer(kafka, async=True)
topic_name = "final"

In [3]:
def get_all_posts(fanpageID):
    """Get all posts in fanpage and write each column in dicts.
       Returns:
            postID_list:
            fanpageName_list:
            content_list:
            createTime_list:
    """
    all_posts = graph.get_connections(id = fanpageID, connection_name = 'posts', summary='true', limit = 100)
    fanpageName = graph.get_object(id = fanpageID, fields = 'name')['name']
    postID_list = []
    fanpageName_list = []
    content_list = []
    createTime_list = []

    for i in range(0,99):
        postID_list.append(all_posts['data'][i]['id'])
        fanpageName_list.append(fanpageName)
        try:
            content_list.append(all_posts['data'][i]['message'])
        except:
            content_list.append('')
        createTime_list.append(all_posts['data'][i]['created_time'])

    return postID_list, fanpageName_list, content_list, createTime_list

In [4]:
def get_activation(postID_list):
    """Get likes and comments in each post and write total counts in dicts.
       Returns:
            postLikesCount_list:
            postCommentsCount_list:
    """
    url = 'https://graph.facebook.com/v3.0/'+ fanpageID +'?fields=posts.limits(100){likes,comments,shares}&access_token=' + token
    response = requests.get(url)
    html = json.loads(response.text)

    postLikesCount_list = []
    postCommentsCount_list = []
    postSharesCount_list = []
    for i in postID_list:
        postLikesCount = graph.get_connections(id = i, connection_name = 'likes', summary='true')
        postCommentsCount = graph.get_connections(id = i, connection_name = 'comments', summary='true')
        postLikesCount_list.append(postLikesCount['summary']['total_count'])
        postCommentsCount_list.append(postCommentsCount['summary']['total_count'])
        
    for i in range(100):
        try:
            postSharesCount = html['posts']['data'][i]['shares']['count']           
        except:
            postSharesCount = 0
        postSharesCount_list.append(postSharesCount)    
    return postLikesCount_list, postCommentsCount_list, postSharesCount_list

In [5]:
def write_file(write_list_1, write_list_2, write_list_3, write_list_4, write_list_5, write_list_6, write_list_7):
    """Write file for list.
    Args:
    filename(str): write out file name.
        write_list(list): list to write out.
    """
    lists = write_list_1, write_list_2, write_list_3, write_list_4, write_list_5, write_list_6, write_list_7
    for i in range(0,99):
        write_out = [li[i] for li in lists]
        str1 = ''.join(str(write_out))
        producer.send_messages(topic_name,str1.encode('UTF-8'))

In [6]:
if __name__ == '__main__':
    token = 'EAACEdEose0cBACZArDykZB19FJO7blBZBJKN98j12KH3ZBHrNNIDbd7Jl25IIN3O1uJXuRYe9PmlVFfVziNdH60vgoKDRMovwIqkllXhzWfC1lp3ZBW5lRBtdPChoUjmgyDVGFYgqqU9juHv0pnJXjCj52H9yzax4zNdvwWQlvUTwZAaj0hWMiElYmAQ9KLm4NqvywgaCVEQZDZD' #貼上token
    graph = facebook.GraphAPI(access_token = token)
    fanpageID = '1021581417961885'

    postID_list, fanpageName_list, content_list, createTime_list = get_all_posts(fanpageID)
    postLikesCount_list, postCommentsCount_list, postSharesCount_list = get_activation(postID_list)
    write_file(postID_list, fanpageName_list, content_list, createTime_list, postLikesCount_list, postCommentsCount_list, postSharesCount_list)